### Evaluation

There are 3 sets of configuration that are set.  One for the prediction (`c_p`), one for the  (`c_r`), one for the experiment(`c_e`). 

I've generated some fake predictions so that for the evaluation time period selected the results are initial 100% correct. 

In [3]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [4]:
#Set config. 
import sys, datetime, os, importlib
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', 9999)
sys.path.append(os.path.join(Path.cwd().parent, 'modules'))
import evaluation as ev
importlib.reload(ev)
#These are configuration prediction files
c_p={'dir': '../data/predictions/tests/',
         'file': 'tests_75_shift0.csv',
         'per_col': 'person_id',
         'date_col': 'yyyymm',
         'date_for': '%Y%m'
        }
#These are configuration referral files
c_r={ 'dir': '../data/referrals/',
         'file': 'test.csv',
         'per_col': 'person_id',
         'date_col_in': 'date',
         'date_for_in': '%m/%d/%Y',
         'date_col_out': 'yyyymm', 
         'date_for_out': '%Y%m', 
         'reason_col': 'class'
        }
#These are configuration for the experiment
c_e={    'experiment': 'Initial Tests',
         'dir': '../results/',
         'file': 'results.csv',
         'save':  True,
         'append': False,
         'thresholds': [0.5, 0.6],
         'k': [5, 10], #k is used the set the conversion from prob to class prediction
         'landmarks': [[0,3], [0,6], [0,12]],
         'ref_target': 'ref',
         'pred_target': 'pref',
         'eval_date': pd.to_datetime(datetime.date(2017,1,1))
        }

In [24]:
#generate fake datasets. This will take a while.  Generate a number of face datasets with the truth "shifted" to various degrees. 
#

importlib.reload(ev)
numberofpatients=20
startedate='2016-01-01'
enddate='2018-01-01'
ev.generate_test_prediction_files(c_p, c_r, 1000,numberofpatients, startedate, enddate)

shift-12
Saving dataframe.  Records: 500 Patients 20
shift-6
Saving dataframe.  Records: 500 Patients 20
shift0
Saving dataframe.  Records: 500 Patients 20
shift6
Saving dataframe.  Records: 500 Patients 20
shift12
Saving dataframe.  Records: 500 Patients 20


In [14]:
#This is the truth 
ref=pd.read_csv(c_r['dir']+c_r['file'])
ref

,person_id,date,class
0,3000,1/1/2017,diabetes
1,3001,1/2/2017,liver
2,3002,2/2/2017,liver
3,3003,3/2/2017,liver
4,3004,4/2/2017,liver
5,3005,5/3/2017,liver
6,3006,6/3/2017,liver
7,3007,7/4/2017,liver
8,3008,8/4/2017,liver
9,3009,9/4/2017,liver


In [15]:
#The above referrals are preprocessed. 
ref, trans = ev.preprocess_referrals(c_r)
print("the label encoding is:", trans)
ref

the label encoding is: {'diabetes': 1, 'liver': 2, 'pnemonia': 3}


,person_id,yyyymm,lab_1,lab_2,lab_3,ref
0,1005,201701,1,0,0,1
1,1006,201702,0,1,0,1
2,1007,201702,0,1,0,1
3,1007,201706,0,1,0,1
4,2000,201710,0,0,1,1
5,2001,201710,0,0,1,1
6,3000,201701,1,0,0,1
7,3001,201701,0,1,0,1
8,3002,201702,0,1,0,1
9,3003,201703,0,1,0,1


In [11]:
#These are the predictions. 
pred=pd.read_csv(c_p['dir']+c_p['file'])

pred

,person_id,yyyymm,pref,plab_1,plab_2,plab_3
0,0.0,201601,0.0,0.0,0.0,0.0
1,0.0,201602,0.0,0.0,0.0,0.0
2,0.0,201603,0.0,0.0,0.0,0.0
3,0.0,201604,0.0,0.0,0.0,0.0
4,0.0,201605,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
249995,9999.0,201709,0.0,0.0,0.0,0.0
249996,9999.0,201710,0.0,0.0,0.0,0.0
249997,9999.0,201711,0.0,0.0,0.0,0.0
249998,9999.0,201712,0.0,0.0,0.0,0.0


## Run One Evaluation
This runs one evaluation at the time in the config. 

In [5]:
#This runs the evaluation at the time set in the config.
importlib.reload(ev)
results = ev.score_times(c_p, c_r, c_e)
results

Loading reference dataframe.. test.csv
Loading predictions dataframe.. tests_75_shift0.csv
Shape of referrals dataframe: (2500, 6)
Shape of referrals dataframe: (100, 3)
Splitting dataset for evaluation at 2017-01-01 00:00:00 Evaluating from: 201701-201704
Examining Columns Slice: DatetimeIndex(['2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01'], dtype='datetime64[ns]', name='datetime', freq=None)


ValueError: y_true contains only one label (0). Please provide the true labels explicitly through the labels argument.

# Multiple different evaluations
This is an example of multiple different evaluations. This evaluates a bunch of files where the predictions have been shifted. 


In [ ]:
importlib.reload(ev)
c_e['experiment']='All Tests'
c_e['append']= True
c_e['file']= 'all_results.csv'

pred_files = [f for f in os.listdir(c_p['dir']) if os.path.isfile(os.path.join(c_p['dir'], f))]

for file in pred_files:
    c_p['file']=file
    results = ev.score_times(c_p, c_r, c_e)

In [ ]:
#read in the full results
results=pd.read_csv(c_e['dir']+c_e['file'])
results